In [ ]:
import cv2
import numpy as np


net = cv2.dnn.readNetFromCaffe('deploy.prototxt', 'deploy.caffemodel')

# Captura (0 es default camera)
cap = cv2.VideoCapture(0)

# contar y seguir
person_count = 0
trackers = []

# nuevo tracker para persona detectada
def create_tracker(frame, bbox):
    tracker = cv2.TrackerKCF_create()
    tracker.init(frame, tuple(bbox))
    return tracker

# main
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # frame en tamaño fijado
    frame_resized = cv2.resize(frame, (300, 300))

    # detectar con frame
    blob = cv2.dnn.blobFromImage(frame_resized, 0.007843, (300, 300), 127.5)
    net.setInput(blob)
    detections = net.forward()

    # lista de cajas de personas detectadas
    bboxes = []

    # Loop de cada detección
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        # Detecciones error borradas
        if confidence > 0.5:
            idx = int(detections[0, 0, i, 1])

            # Checkear si detección es persona (class label 15 for MobileNet-SSD)
            if idx == 15:
                box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
                (startX, startY, endX, endY) = box.astype("int")
                bboxes.append([startX, startY, endX - startX, endY - startY])

    # Update trackers y contar personas
    new_trackers = []
    for tracker in trackers:
        success, bbox = tracker.update(frame)
        if success:
            new_trackers.append(tracker)

    # Checkear nuevas personas a trackear
    for bbox in bboxes:
        matched = False
        for tracker in new_trackers:
            success, tracked_bbox = tracker.update(frame)
            if success:
                iou = calculate_iou(bbox, tracked_bbox)
                if iou > 0.5:
                    matched = True
                    break
        if not matched:
            tracker = create_tracker(frame, bbox)
            new_trackers.append(tracker)
            person_count += 1

    trackers = new_trackers

    # Display
    for bbox in bboxes:
        (x, y, w, h) = [int(v) for v in bbox]
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.putText(frame, f'Person Count: {person_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow("Frame", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()